In [279]:
import pandas as pd
import nltk
import seaborn as sns #Kenneth
import matplotlib.pyplot as plt
import string
import demoji
import contractions
import time

from langdetect import detect,detect_langs #pip install langdetect (Kenneth)
from wordcloud import WordCloud, ImageColorGenerator
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.pipeline import Pipeline
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

import warnings
warnings.filterwarnings("ignore")
#nltk.download('wordnet')
#nltk.download('omw-1.4')
#nltk.download('punkt')



In [280]:
df = pd.read_csv('clean_dataset.csv')

In [281]:
df_temp = df

In [282]:
import emoji

def convert_demojis(text):
    text_temp = emoji.demojize(text)
    return text_temp.replace(":", "")

df_temp['demojiReview'] = df_temp['Product Review'].apply(convert_demojis)


In [283]:
df_temp

,No,Author,Product Rating,Product Review,demojiReview
0,1,n*****n,3,"baju kurta sampai dalam keadaan baik, tapi tak...","baju kurta sampai dalam keadaan baik, tapi tak..."
1,2,y*****i,2,"Sizing tak accurate langsung, aku rasa macam b...","Sizing tak accurate langsung, aku rasa macam b..."
2,3,amalinadinee,2,"Baju kurta aku dah koyak sebelum pakai, memang...","Baju kurta aku dah koyak sebelum pakai, memang..."
3,4,c*****8,5,Delivery laju. Kain cantik. Thanks seller. Mas...,Delivery laju. Kain cantik. Thanks seller. Mas...
4,5,8kbvdmoyw9,5,Baju selamat mendarat kain cantik dan sedap di...,Baju selamat mendarat kain cantik dan sedap di...
...,...,...,...,...,...
457,9,s*****a,3,Pada baju ada warna merah\nCuba basuh masih x ...,Pada baju ada warna merah\nCuba basuh masih x ...
458,11,*****n,2,Colour a bit different from the picture. And m...,Colour a bit different from the picture. And m...
459,12,m*****4,1,Warna nude kita mintak warna lain yg Smpai lpa...,Warna nude kita mintak warna lain yg Smpai lpa...
460,14,k_w1z356vg,5,tidak mengikuti saiz yg saya nak,tidak mengikuti saiz yg saya nak


In [340]:
from googletrans import Translator
import pandas as pd

# Function to translate text
def translate_text(text, source_lang='ms', target_lang='en'):
    try:
        # Initialize the translator
        translator = Translator()

        # Translate text
        translation = translator.translate(text, src=source_lang, dest=target_lang)

        return translation.text
    except Exception as e:
        return str(e)

        
# Translate text in a specific column (e.g., 'demojiReview') and store the translations in a new column
df_temp['translated_text'] = df_temp['demojiReview'].apply(translate_text)

# Display the translated text
print(df_temp['translated_text'])


not good
0      Kurta's clothes are in good condition, but no ...
1      The sizing is not accurate at all, I feel like...
2      My shirt is already torn before wearing it, it...
3      Delivery speed.Beautiful cloth.Thanks seller.S...
4      The clothes are safe to land a beautiful and t...
                             ...                        
457    On the shirt there is a red color\nTry to wash...
458    Color a bit different from the picture.And mod...
459    Nude color we ask for other colors with lpa tu...
460                        Not following the size I want
461                        The size is not the same size
Name: translated_text, Length: 462, dtype: object


In [285]:
df_temp

,No,Author,Product Rating,Product Review,demojiReview,translated_text
0,1,n*****n,3,"baju kurta sampai dalam keadaan baik, tapi tak...","baju kurta sampai dalam keadaan baik, tapi tak...","Kurta's clothes are in good condition, but no ..."
1,2,y*****i,2,"Sizing tak accurate langsung, aku rasa macam b...","Sizing tak accurate langsung, aku rasa macam b...","The sizing is not accurate at all, I feel like..."
2,3,amalinadinee,2,"Baju kurta aku dah koyak sebelum pakai, memang...","Baju kurta aku dah koyak sebelum pakai, memang...","My shirt is already torn before wearing it, it..."
3,4,c*****8,5,Delivery laju. Kain cantik. Thanks seller. Mas...,Delivery laju. Kain cantik. Thanks seller. Mas...,Delivery speed.Beautiful cloth.Thanks seller.S...
4,5,8kbvdmoyw9,5,Baju selamat mendarat kain cantik dan sedap di...,Baju selamat mendarat kain cantik dan sedap di...,The clothes are safe to land a beautiful and t...
...,...,...,...,...,...,...
457,9,s*****a,3,Pada baju ada warna merah\nCuba basuh masih x ...,Pada baju ada warna merah\nCuba basuh masih x ...,On the shirt there is a red color\nTry to wash...
458,11,*****n,2,Colour a bit different from the picture. And m...,Colour a bit different from the picture. And m...,Color a bit different from the picture.And mod...
459,12,m*****4,1,Warna nude kita mintak warna lain yg Smpai lpa...,Warna nude kita mintak warna lain yg Smpai lpa...,Nude color we ask for other colors with lpa tu...
460,14,k_w1z356vg,5,tidak mengikuti saiz yg saya nak,tidak mengikuti saiz yg saya nak,Not following the size I want


In [342]:
# https://www.geeksforgeeks.org/nlp-expand-contractions-in-text-processing/
# https://www.aasmr.org/jsms/Vol12/JSMS%20OCT%202022/Vol.12.No.05.05.pdf
import contractions

# Define additional contractions
contractions.add("x", "not")

def expand_contractions(text):
    expanded_words = [] 
    for word in text.split():
        expanded_words.append(contractions.fix(word))
    expanded_text = ' '.join(expanded_words)
    return expanded_text

# Apply the function to your DataFrame
df_temp['ContractionReview'] = df_temp['translated_text'].apply(expand_contractions)
df_temp['ContractionReview'].head()



0    Kurta's clothes are in good condition, but no ...
1    The sizing is not accurate at all, I feel like...
2    My shirt is already torn before wearing it, it...
3    Delivery speed.Beautiful cloth.Thanks seller.S...
4    The clothes are safe to land a beautiful and t...
Name: ContractionReview, dtype: object

In [287]:
df_temp

,No,Author,Product Rating,Product Review,demojiReview,translated_text,ContractionReview
0,1,n*****n,3,"baju kurta sampai dalam keadaan baik, tapi tak...","baju kurta sampai dalam keadaan baik, tapi tak...","Kurta's clothes are in good condition, but no ...","Kurta's clothes are in good condition, but no ..."
1,2,y*****i,2,"Sizing tak accurate langsung, aku rasa macam b...","Sizing tak accurate langsung, aku rasa macam b...","The sizing is not accurate at all, I feel like...","The sizing is not accurate at all, I feel like..."
2,3,amalinadinee,2,"Baju kurta aku dah koyak sebelum pakai, memang...","Baju kurta aku dah koyak sebelum pakai, memang...","My shirt is already torn before wearing it, it...","My shirt is already torn before wearing it, it..."
3,4,c*****8,5,Delivery laju. Kain cantik. Thanks seller. Mas...,Delivery laju. Kain cantik. Thanks seller. Mas...,Delivery speed.Beautiful cloth.Thanks seller.S...,Delivery speed.Beautiful cloth.Thanks seller.S...
4,5,8kbvdmoyw9,5,Baju selamat mendarat kain cantik dan sedap di...,Baju selamat mendarat kain cantik dan sedap di...,The clothes are safe to land a beautiful and t...,The clothes are safe to land a beautiful and t...
...,...,...,...,...,...,...,...
457,9,s*****a,3,Pada baju ada warna merah\nCuba basuh masih x ...,Pada baju ada warna merah\nCuba basuh masih x ...,On the shirt there is a red color\nTry to wash...,On the shirt there is a red color\nTry to wash...
458,11,*****n,2,Colour a bit different from the picture. And m...,Colour a bit different from the picture. And m...,Color a bit different from the picture.And mod...,Color a bit different from the picture.And mod...
459,12,m*****4,1,Warna nude kita mintak warna lain yg Smpai lpa...,Warna nude kita mintak warna lain yg Smpai lpa...,Nude color we ask for other colors with lpa tu...,Nude color we ask for other colors with lpa tu...
460,14,k_w1z356vg,5,tidak mengikuti saiz yg saya nak,tidak mengikuti saiz yg saya nak,Not following the size I want,Not following the size I want Not following th...


In [415]:
# import re  
# def remove_special_characters(text):
#     return re.sub(r'[^a-zA-Z0-9\s]', '', text)

# df_temp['RemovedSpecialCharReview'] = df_temp['ContractionReview'].apply(remove_special_characters)

# df_temp['RemovedSpecialCharReview'].head()

0    Kurtas clothes are in good condition but no st...
1    The sizing is not accurate at all I feel like ...
2    My shirt is already torn before wearing it it ...
3    Delivery speedBeautiful clothThanks sellerStil...
4    The clothes are safe to land a beautiful and t...
Name: RemovedSpecialCharReview, dtype: object

In [446]:
remove = set(string.punctuation)

def remove_punctuation(text):
    return ' '.join(word.strip(''.join(remove)) for word in text.split())

df_temp['RemovedPuncReview'] = df_temp['ContractionReview'].apply(remove_punctuation)

df_temp['RemovedPuncReview'].head()

0    Kurta's clothes are in good condition but no s...
1    The sizing is not accurate at all I feel like ...
2    My shirt is already torn before wearing it it ...
3    Delivery speed.Beautiful cloth.Thanks seller.S...
4    The clothes are safe to land a beautiful and t...
Name: RemovedPuncReview, dtype: object

In [447]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download NLTK stopwords if you haven't already
#nltk.download('stopwords')
#nltk.download('punkt')

# Load stopwords for the desired language
stop_words = set(stopwords.words('english'))

# Function to remove stopwords from text
def remove_stopwords(text):
    # Tokenize the text
    word_tokens = word_tokenize(text)
    # Remove stopwords
    filtered_sentence = [word for word in word_tokens if word.lower() not in stop_words]
    # Join the words back into a sentence
    filtered_text = ' '.join(filtered_sentence)
    return filtered_text


# Apply stopwords removal to each text entry
df_temp['stopwords_removal'] = df_temp['RemovedPuncReview'].apply(remove_stopwords)


df_temp['stopwords_removal'].head()

0             Kurta 's clothes good condition standout
1    sizing accurate feel like low school clothes.E...
2    shirt already torn wearing good quality ! face...
3    Delivery speed.Beautiful cloth.Thanks seller.S...
4    clothes safe land beautiful tasty fabric frien...
Name: stopwords_removal, dtype: object

In [448]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download NLTK resources if you haven't already
#nltk.download('punkt')
#nltk.download('wordnet')

# Initialize the WordNet lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to lemmatize text
def lemmatize_text(text):
    words = word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

# Apply lemmatization to the 'text' column
df_temp['lemma_text'] = df_temp['stopwords_removal'].apply(lemmatize_text)

# Display the lemmatized text
df_temp['lemma_text']


0               Kurta 's clothes good condition standout
1      sizing accurate feel like low school clothes.E...
2      shirt already torn wearing good quality ! face...
3      Delivery speed.Beautiful cloth.Thanks seller.S...
4      clothes safe land beautiful tasty fabric frien...
                             ...                        
457    shirt red color Try wash still lost red impres...
458    Color bit different picture.And modern size cu...
459    Nude color ask color lpa tue button free butto...
460                                  following size want
461                                            size size
Name: lemma_text, Length: 462, dtype: object

In [449]:
import pandas as pd
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Create Stemmer object
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Perform lemmatization on each sentence in the 'text' column
df_temp['stemming_text'] = df_temp['stopwords_removal'].apply(lambda x: stemmer.stem(x))

# Display the original and lemmatized text
df_temp['stemming_text'].head()

0              kurta s clothes good condition standout
1    sizing accurate feel like low school clothes e...
2    shirt already torn wearing good quality face w...
3    delivery speed beautiful cloth thanks seller s...
4    clothes safe land beautiful tasty fabric frien...
Name: stemming_text, dtype: object

In [174]:
#preprocessing 

In [450]:
# Function to map ratings to sentiments
def map_sentiment(rating):
    if rating in [1, 2]:
        return 'Negative'
    elif rating == 3:
        return 'Neutral'
    elif rating in [4, 5]:
        return 'Positive'
    else:
        return 'Unknown'  # Handle any other cases if needed

# Apply mapping to create 'Sentiment_Rating' column
df_temp['Sentiment_Rating'] = df_temp['Product Rating'].apply(map_sentiment)

# Display the DataFrame with the new 'Sentiment_Rating' column
print(df_temp[['translated_text','Product Rating', 'Sentiment_Rating']])

                                       translated_text  Product Rating  \
0    Kurta's clothes are in good condition, but no ...               3   
1    The sizing is not accurate at all, I feel like...               2   
2    My shirt is already torn before wearing it, it...               2   
3    Delivery speed.Beautiful cloth.Thanks seller.S...               5   
4    The clothes are safe to land a beautiful and t...               5   
..                                                 ...             ...   
457  On the shirt there is a red color\nTry to wash...               3   
458  Color a bit different from the picture.And mod...               2   
459  Nude color we ask for other colors with lpa tu...               1   
460                      Not following the size I want               5   
461                      The size is not the same size               1   

    Sentiment_Rating  
0            Neutral  
1           Negative  
2           Negative  
3           Positiv

In [238]:
#convert 

In [239]:
# finalData = pd.read_csv('Contraction Review & Sentiment Rating.csv')
# finalData.isnull().any().any()

In [451]:
x = df_temp['lemma_text']
y = df_temp['Sentiment_Rating']

In [452]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


# Print the shapes of training and testing sets
print("Number of samples in Training Set:", len(X_train))
print("Number of samples in Testing Set:", len(X_test))


Number of samples in Training Set: 369
Number of samples in Testing Set: 93


In [453]:
# Tokenize the text data
tokenized_data_train = X_train.apply(lambda x: word_tokenize(x.lower()))
tokenized_data_test = X_test.apply(lambda x: word_tokenize(x.lower()))

In [454]:
from nltk.tokenize import word_tokenize

#convert it into tf-idf 
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # You can adjust max_features based on your dataset size

x_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
x_test_tfidf = tfidf_vectorizer.transform(X_test)

print(x_train_tfidf)
print('===================')
print(x_test_tfidf)


  (0, 361)	0.4951817801589717
  (0, 171)	0.39632278143433564
  (0, 200)	0.6371881891965532
  (0, 83)	0.3411101514911258
  (0, 643)	0.18090279852654328
  (0, 423)	0.20647641777725398
  (1, 206)	0.37799507079764394
  (1, 670)	0.5206970976215178
  (1, 48)	0.3859733497568338
  (1, 65)	0.2742223180929166
  (1, 199)	0.48352603916927517
  (1, 83)	0.27874820792680544
  (1, 643)	0.14783005043322422
  (1, 423)	0.16872828669261483
  (2, 69)	0.3450058251402657
  (2, 196)	0.24756899605085605
  (2, 266)	0.2953519263855258
  (2, 495)	0.3565773194931645
  (2, 406)	0.4451334058640082
  (2, 581)	0.5488230071488391
  (2, 305)	0.2795849705537046
  (2, 643)	0.16668002179500066
  (3, 634)	0.22642253143395163
  (3, 485)	0.45976666975307545
  (3, 756)	0.408533913972458
  :	:
  (368, 727)	0.1598786003293344
  (368, 576)	0.19164999216111195
  (368, 404)	0.1598786003293344
  (368, 309)	0.19164999216111195
  (368, 794)	0.17224591891748217
  (368, 143)	0.16353186245468207
  (368, 93)	0.17224591891748217
  (368, 49

In [455]:
from gensim.models import Word2Vec


# Train Word2Vec model on tokenized data
word2vec_model_train = Word2Vec(sentences=tokenized_data_train, vector_size=100, window=5, min_count=1, workers=4)
word2vec_model_test  = Word2Vec(sentences=tokenized_data_test, vector_size=100, window=5, min_count=1, workers=4)

x_train_w2v = []
for doc_tokens in tokenized_data_train:
    doc_vector = sum(word2vec_model_train.wv[token] for token in doc_tokens) / len(doc_tokens)
    x_train_w2v.append(doc_vector)

x_test_w2v = []
for doc_tokens in tokenized_data_test:
    doc_vector = sum(word2vec_model_test.wv[token] for token in doc_tokens) / len(doc_tokens)
    x_test_w2v.append(doc_vector)

    
print(x_train_w2v)
print('===================')
print(x_test_w2v)


[array([-4.1875374e-03,  1.2626334e-03,  2.1942446e-03,  4.2414931e-03,
        2.1419321e-04, -4.5068902e-03, -9.1967121e-04,  6.4056353e-03,
       -3.0396143e-03,  2.0787042e-05, -9.7393169e-04, -5.1223058e-03,
       -5.6494982e-03,  4.7208625e-03, -7.7931210e-05, -2.9036908e-03,
       -4.5545222e-04,  1.3410486e-03, -3.4591921e-03, -4.9171024e-03,
        3.6620640e-03,  1.7285471e-04,  3.8025582e-03, -1.9822477e-03,
        8.9421048e-04,  2.6154419e-04,  5.4457537e-06,  2.8034978e-04,
       -1.2989356e-03, -2.8641531e-04,  2.4151288e-03, -9.2843710e-04,
        5.8436473e-03, -7.1699050e-04, -6.5817311e-04,  6.6112523e-04,
        1.4033183e-03, -1.1135199e-03, -4.6442728e-06, -9.5408992e-04,
        6.0503394e-04, -4.8251930e-04, -4.1921283e-03,  1.0396240e-03,
        2.9249527e-03,  8.0614444e-04, -9.9957257e-04, -2.7656883e-03,
        5.3456076e-03,  2.2380869e-03,  2.9602539e-04, -1.5716132e-03,
        4.6599251e-03, -8.2389149e-04,  2.5101882e-04,  1.9291313e-03,
     

In [456]:
import spacy
import numpy as np

# Load pre-trained GloVe vectors using spaCy
nlp = spacy.load('en_core_web_md') 

x_train_glo = []
for text in tokenized_data_train:
    doc = nlp(' '.join(text))  # Concatenate tokens into a single string
    doc_vector = np.mean([token.vector for token in doc], axis=0)
    x_train_glo.append(doc_vector)

x_test_glo = []
for text in tokenized_data_test:
    doc = nlp(' '.join(text))
    doc_vector = np.mean([token.vector for token in doc], axis=0)
    x_test_glo.append(doc_vector)

    
    
print(x_train_glo)
print('===========================')
print(x_test_glo)

[array([-1.8519001e+00, -7.5399828e-01, -1.3806852e+00, -9.7912198e-01,
        1.2612666e+00, -5.2792007e-01,  2.9760168e+00,  2.4940317e+00,
       -1.9525701e+00, -3.2310334e-01,  3.9246416e+00,  2.0487866e+00,
       -4.4017000e+00,  2.4838681e+00,  1.0147017e+00,  3.2852831e-01,
        1.2623334e+00,  8.1252354e-01,  2.4954174e-01, -6.9014996e-02,
       -4.3569672e-01,  1.4406749e+00, -2.0361083e+00, -9.2908841e-01,
       -2.8436932e-01, -2.0437868e+00, -6.5072334e-01, -1.9012151e+00,
        2.2749981e-02,  4.7918335e-01,  1.4706267e+00, -1.8646201e+00,
        2.2139998e-01, -1.3839468e+00,  1.6301495e+00, -8.7150002e-01,
        1.2184349e+00,  1.6121167e+00,  5.9839666e-01, -1.0360450e+00,
       -1.5049237e+00, -1.5013231e-01,  7.0788002e-01, -1.2469983e+00,
       -6.2941003e-01,  1.9120184e+00,  6.6327667e-01, -1.4716567e+00,
       -1.2541665e-01,  6.3483320e-02,  1.8908316e+00,  8.5528415e-01,
        4.7209033e-01, -2.2561483e+00, -3.0169335e-01,  1.0005549e+00,
     

In [457]:
def min_max_scaling(embeddings):
    min_vals = np.min(embeddings, axis=0)
    max_vals = np.max(embeddings, axis=0)
    normalized_embeddings = (embeddings - min_vals) / (max_vals - min_vals)
    return normalized_embeddings


x_train_w2v = min_max_scaling(x_train_w2v)
x_train_glo = min_max_scaling(x_train_glo)

x_test_w2v = min_max_scaling(x_test_w2v)
x_test_glo = min_max_scaling(x_test_glo)

In [458]:
#modelling 

In [459]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  
    'penalty': ['l1', 'l2'],  
    'solver': ['liblinear', 'saga']  
}

logistic_regression = LogisticRegression()

grid_search_tfidf_temp = GridSearchCV(logistic_regression, param_grid, cv=5, scoring='accuracy')
grid_search_w2v_temp   = GridSearchCV(logistic_regression, param_grid, cv=5, scoring='accuracy')
grid_search_glo_temp   = GridSearchCV(logistic_regression, param_grid, cv=5, scoring='accuracy')


grid_search_tfidf = grid_search_tfidf_temp.fit(x_train_tfidf, y_train)
grid_search_w2v   = grid_search_w2v_temp.fit(x_train_w2v, y_train)
grid_search_glo   = grid_search_glo_temp.fit(x_train_glo, y_train)

best_params_tfidf = grid_search_tfidf.best_params_
best_params_w2v   = grid_search_w2v.best_params_
best_params_glo   = grid_search_glo.best_params_


print("Best Parameters for TF-IDF :", best_params_tfidf)
print("Best Parameters for W2V    :", best_params_w2v)
print("Best Parameters for GloVe  :", best_params_glo)


Best Parameters for TF-IDF : {'C': 1, 'penalty': 'l2', 'solver': 'saga'}
Best Parameters for W2V    : {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}
Best Parameters for GloVe  : {'C': 1, 'penalty': 'l1', 'solver': 'saga'}


In [460]:
logistic_regression_tfidf   = LogisticRegression(**best_params_tfidf)
logistic_regression_w2v     = LogisticRegression(**best_params_w2v)
logistic_regression_glo     = LogisticRegression(**best_params_glo)

logistic_regression_tfidf.fit(x_train_tfidf, y_train)
logistic_regression_w2v.fit(x_train_w2v, y_train)
logistic_regression_glo.fit(x_train_glo, y_train)

lr_predictions_tfidf = logistic_regression_tfidf.predict(x_test_tfidf)
lr_predictions_w2v   = logistic_regression_w2v.predict(x_test_w2v)
lr_predictions_glo   = logistic_regression_glo.predict(x_test_glo)

In [461]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

accuracy_lr_tfidf  = accuracy_score(y_test, lr_predictions_tfidf)
precision_lr_tfidf = precision_score(y_test, lr_predictions_tfidf, average='micro')
recall_lr_tfidf    = recall_score(y_test, lr_predictions_tfidf, average='micro')
f1_lr_tfidf        = f1_score(y_test, lr_predictions_tfidf, average='micro')

accuracy_lr_w2v  = accuracy_score(y_test, lr_predictions_w2v)
precision_lr_w2v = precision_score(y_test, lr_predictions_w2v, average='micro')
recall_lr_w2v    = recall_score(y_test, lr_predictions_w2v, average='micro')
f1_lr_w2v        = f1_score(y_test, lr_predictions_w2v, average='micro')

accuracy_lr_glo  = accuracy_score(y_test, lr_predictions_glo)
precision_lr_glo = precision_score(y_test, lr_predictions_glo, average='micro')
recall_lr_glo    = recall_score(y_test, lr_predictions_glo, average='micro')
f1_lr_glo        = f1_score(y_test, lr_predictions_glo, average='micro')

print('Logistic Regression Model Metrics - TF-IDF:')
print('Accuracy  : {:.2f} %'.format(accuracy_lr_tfidf*100))
# print('Precision : {:.2f} %'.format(precision_lr_tfidf*100))
# print('Recall    : {:.2f} %'.format(recall_lr_tfidf*100))
# print('F1 Score  : {:.2f} %'.format(f1_lr_tfidf*100))
print('')
print('Logistic Regression Model Metrics - W2V:')
print('Accuracy  : {:.2f} %'.format(accuracy_lr_w2v*100))
# print('Precision : {:.2f} %'.format(precision_lr_w2v*100))
# print('Recall    : {:.2f} %'.format(recall_lr_w2v*100))
# print('F1 Score  : {:.2f} %'.format(f1_lr_w2v*100))
print('')
print('Logistic Regression Model Metrics - GloVe:')
print('Accuracy  : {:.2f} %'.format(accuracy_lr_glo*100))
# print('Precision : {:.2f} %'.format(precision_lr_glo*100))
# print('Recall    : {:.2f} %'.format(recall_lr_glo*100))
# print('F1 Score  : {:.2f} %'.format(f1_lr_glo*100))

Logistic Regression Model Metrics - TF-IDF:
Accuracy  : 77.42 %

Logistic Regression Model Metrics - W2V:
Accuracy  : 29.03 %

Logistic Regression Model Metrics - GloVe:
Accuracy  : 63.44 %


In [ ]:
def predict_sentiment(text):
    prediction = knn.predict([text])
    confidence = knn.predict_proba([text])
    confidence_level = np.max(confidence) * 100
    return prediction, confidence_level

# Example usage of the function
user_input = input("Enter your text: ")
sentiment, confidence_level = predict_sentiment(user_input)
print("Predicted sentiment:", sentiment)
print("Confidence level:", confidence_level)

In [477]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import contractions
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

class ContractionExpander(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        expanded_texts = []
        for text in X:
            # Expand contractions
            expanded_text = contractions.fix(text)
            # Insert 'x' for 'not' if not already present
            expanded_text = expanded_text.replace(" x ", " not ") if "not " in expanded_text else expanded_text
            expanded_texts.append(expanded_text)
        return expanded_texts

# Define the pipeline
logistic_regression_pipeline = Pipeline([
    ('contraction_expander', ContractionExpander()), 
    ('tfidf', TfidfVectorizer(max_features=1000)),  # You can customize TfidfVectorizer parameters here
    ('logistic_regression', LogisticRegression(**{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'})),  # You can customize LogisticRegression parameters here
])


X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Train the pipeline (Assuming X_train and y_train are your training data)
logistic_regression_pipeline.fit(X_train, y_train)

# Make predictions and get probability estimates
y_pred = logistic_regression_pipeline.predict(X_test)
confidence = logistic_regression_pipeline.predict_proba(X_test)

accuracy_lr_tfidf  = accuracy_score(y_test, y_pred)
precision_lr_tfidf = precision_score(y_test, y_pred, average='micro')
recall_lr_tfidf    = recall_score(y_test, y_pred, average='micro')
f1_lr_tfidf        = f1_score(y_test, y_pred, average='micro')

print('Logistic Regression Model Metrics - TF-IDF:')
print('Accuracy  : {:.2f} %'.format(accuracy_lr_tfidf*100))
# print('Precision : {:.2f} %'.format(precision_lr_tfidf*100))
# print('Recall    : {:.2f} %'.format(recall_lr_tfidf*100))
# print('F1 Score  : {:.2f} %'.format(f1_lr_tfidf*100))

# Calculate overall confidence level
overall_confidence = np.mean([max(conf) for conf in confidence]) * 100
print("Overall confidence level: {:.2f} %".format(overall_confidence))


def predict_sentiment(text):
    prediction = logistic_regression_pipeline.predict([text])
    confidence = logistic_regression_pipeline.predict_proba([text])
    confidence_level = np.max(confidence) * 100
    return prediction, confidence_level

# Example usage of the chatbot
user_input = input("Enter your text: ")
sentiment, confidence_level = predict_sentiment(user_input)
print("Predicted sentiment:", sentiment)
print("Confidence level:", confidence_level)


Logistic Regression Model Metrics - TF-IDF:
Accuracy  : 78.49 %
Overall confidence level: 57.69 %


Enter your text: saya not feeling lucky today
Predicted sentiment: ['Negative']
Confidence level: 50.6209745540845


In [ ]:
# saya not feeling lucky today
# that parcel buruk and x betul
# baju ni okay je pakai